In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from getAttentionLib import load_and_group_logit_diffs, plot_pooled_probs_plt

In [ ]:
n_img_tokens = 256

methods = ["str", "gn_03o", "gn_10o", "gn_20o", "gn_30o"]
for method in methods:
    for reduction in ["mean", "absmax", "max", "absmean"]:
        print(f"Loading {method} {reduction}")
        is_gaussian_noising = method != "str"
        root = Path(f"vqa_patching/{method}/")
        grouped_logit_diffs = load_and_group_logit_diffs(
            root=root,
            is_gaussian_noising=is_gaussian_noising,
            n_img_tokens=n_img_tokens,
            reduction=reduction,
        )
        print(f"Num samples: {grouped_logit_diffs.shape[0]}")
        fig = plot_pooled_probs_plt(
            pooled_probs=grouped_logit_diffs.mean(dim=0).T,  # mean across samples
            inputs_tokens=["img tokens", "bos token", "text tokens", "final token"],
            cbar_label="Logit Diff",
            # cmax=(cmax := clustered_logit_diffs.abs().max()),
            # cmin=-cmax,
            cmap="RdBu",
            n_img_tokens=1,
            figsize=(6, 2),
            cmax=(cmax := 1),
            cmin=-cmax,
            fraction=0.02,
        )
        # fig.show()
        dir = Path(f"imgs/vqa_patching/{method}/")
        dir.mkdir(parents=True, exist_ok=True)
        fig.savefig(dir / f"{reduction}_aggregate_logit_diffs.png")
        fig.clear()
# fig.savefig("imgs/vqa_patching/str_aggregate_logit_diffs.png")